# ASEを使ってQE用の構造モデルを作成する

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
import importlib
if importlib.util.find_spec("ase") is None:
    !pip install git+https://github.com/minoru-otani/ase.git@qe_rism

In [ ]:
import matplotlib.pyplot as plt
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.constraints import FixAtoms
from ase.io.cube import read_cube_data
from ase.build import fcc111, graphene, add_adsorbate, molecule
from ase.geometry import wrap_positions

## Slabモデルを作成する

ASEにはあらかじめ典型的な表面構造を作る関数が用意されている。用意されている構造をリストする。詳しくは[ase.build.surface](https://wiki.fysik.dtu.dk/ase/_modules/ase/build/surface.html)を参照。

||||||||
|---|---|---|---|---|---|---| 
|Surface|fcc(100)|fcc(110)|fcc(111)|bcc(100)|bcc(110)|bcc(111)|
|Adsorption site|ontop, bridge<br>hollow|ontop, longbridge<br>shortbridge, hollow|ontop, bridge<br>fcc, hcp|ontop, bridge<br>hollow|ontop, longbridge<br>shortbridge, hollow|ontop|
|Surface|hcp(0001)|hcp(10$\bar{1}$0)|diamond(111)|diamond(100)|mx2|graphene|
|Adsorption site|ontop, bridge<br>fcc, hcp|ontop|ontop|ontop|||

### 吸着サイトの情報を見る

吸着サイトの指定の仕方は二通りある：```ontop```などのキーワードを与える方法と、$(x,y)$を直接与える方法である。各表面構造関数が持っている吸着サイトは以下のように調べることができる。例として```fcc(111)```表面に用意されている吸着サイトを表示してみる。

In [ ]:
slab = fcc111('Cu', size=(1,1,5), vacuum=10.0)
slab.info.get('adsorbate_info', {})

このように、```ontop, bridge, fcc```および```hcp```サイトがあることが分かる。もし、あらかじめ用意されている吸着サイトがない場合は何も表示されない。

In [ ]:
slab = graphene(formula='C2', size=(1,1,1), vacuum=10.0)
slab.info.get('adsorbate_info', {})

このような場合は、直接$xy$座標を指定して吸着サイトを指定する必要がある。（詳しくは下で説明します。）

### fcc金属の例

In [ ]:
lvac = 8.0 # 左側の真空の厚み
rvac = 10.0 # 右側の真空の厚み
vac = (lvac + rvac)/2.0 # 表面構造関数には真空の左右の真空の厚みの半分を渡す
slab = fcc111('Cu', size=(4,4,2), vacuum=vac)
slab.wrap() # 普通に作ると、ユニットセルをはみ出す原子があるので、ユニットセル内にwrapする。
slab.translate((0.0,0.0,(lvac-rvac)/2.0)) # lvac, rvacを反映した位置にずらす
fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(slab, ax[0], radii=1.0, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.0, rotation=('90x,90y,90z'))
#view(slab, viewer='ngl')
plt.show()

スラブの中身を確認する。

In [ ]:
aobj = slab
print(f' Unit cell: a = ({aobj.cell[0,0]:9.5f}, {aobj.cell[0,1]:9.5f}, {aobj.cell[0,2]:9.5f})')
print(f'            b = ({aobj.cell[1,0]:9.5f}, {aobj.cell[1,1]:9.5f}, {aobj.cell[1,2]:9.5f})')
print(f'            c = ({aobj.cell[2,0]:9.5f}, {aobj.cell[2,1]:9.5f}, {aobj.cell[2,2]:9.5f})')
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')
    

### 分子吸着を考えるために分子を用意する

In [ ]:
atoms = molecule('CO2')
fig, ax = plt.subplots(1,1, figsize=(3,1.5))
ax.set_axis_off()
plot_atoms(atoms, ax, radii=1.0, rotation=('0x,90y,0z'))
plt.show()
#view(atoms, viewer='ngl')

In [ ]:
atoms = molecule('H2O')
fig, ax = plt.subplots(1,1, figsize=(3,1.5))
ax.set_axis_off()
plot_atoms(atoms, ax, radii=1.0, rotation=('0x,0y,0z'))
plt.show()

### 分子を表面に置く

In [ ]:
atoms = molecule('H2O')
atoms.rotate(180, 'x', center=(0,0,0)) # make o-down orientation
slab = fcc111('Cu', size=(2,2,2), vacuum=20.0)
slab.wrap() # wrap extended atoms into a unit cell.
add_adsorbate(slab,atoms,height=5.0, position='ontop', mol_index=0)
#positions = wrap_positions(positions=slab.positions, cell=slab.cell, pbc=slab.pbc, center=[0.5,0.5,0.0])
#I want to shift the H2O molecule to the center of the unit cell. But I don't understand how to use wrap_positions
fig, ax = plt.subplots(2,1, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(slab, ax[0], radii=1.0, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.0, rotation=('90x,90y,90z'))

### ESM/ESM-RISM計算用にスラブをずらす

In [ ]:
lvac = 10.0
rvac = 10.0
vac = (lvac + rvac)/2.0
atoms = molecule('H2O')
atoms.rotate(180, 'x', center=(0,0,0)) # make o-down orientation
slab = fcc111('Cu', size=(2,2,2), vacuum=vac)
slab.wrap() # wrap extended atoms into a unit cell.
add_adsorbate(slab,atoms,height=5.0, position='ontop', mol_index=0)
slab.translate((0.0,0.0,lvac - vac)) # shift atoms to have given vacuum thickness
slab.translate((0.0,0.0,-slab.cell[2,2]/2.0)) # shift atoms to fit ESM/ESM-RISM model
fig, ax = plt.subplots(1,1, figsize=(12, 6))
ax.set_axis_off()
plot_atoms(slab, ax, radii=1.0, rotation=('90x,90y,90z'))

原子座標などを確認する。

In [ ]:
print('Unit cell:')
print(slab.cell[:])
print('Unit cell size along z-axis: {} A'.format(slab.cell[2,2]))
print('Atom positions:')
print(slab.positions[:])

# ESM-RISM用のインプットファイルを作成する。

In [ ]:
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
input_data = {
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'Al100_bc1',
        'disk_io': 'low',
        'lfcp': True,
        'trism': True,
    },
    'system': {
        'ibrav': 0,
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations' : 'smearing',
        'smearing':'mp',
        'degauss' : 0.03,
        'assume_isolated': 'esm',
        'esm_bc': 'bc1'
    },
    'electrons': {
        'mixing_beta': 0.3,
        'diago_rmm_conv': False
    },
    'fcp': {
        'fcp_mu': -3.5
    }
}
 
solv_info={
    'density_unit' : 'mol/L',
    'H2O':[-1,   'H2O.spc.MOL'],
    'Na+' :[1.00, 'Na+.oplsaa.MOL'],
    'Cl-':[1.00, 'Cl-.oplsaa.MOL']
} 

inpfile = 'test.in'
write(inpfile, slab, format='espresso-in', input_data=input_data, 
      pseudopotentials=pseudopotentials, solvents_info=solv_info)